<div style="max-width:1400px;margin-center: auto">
<img src="images\drivers.png" width="600"/>
</div>

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local') \
    .appName("driver") \
    .config("spark.sql.catalogImplementation", "hive") \
    .getOrCreate()

In [2]:
from pyspark.sql.types import StringType, DateType, IntegerType, StructType, StructField
from pyspark.sql.functions import current_timestamp, concat, lit, col

In [3]:
name_schema = StructType(fields = [StructField("forename", StringType(), True),
                                    StructField("surname", StringType(), True)])

In [4]:
drivers_schema = StructType(fields = [StructField("driverId", IntegerType(), False),
                                      StructField("driverRef", StringType(), False),
                                      StructField("number", IntegerType(), False),
                                      StructField("code", StringType(), False),
                                      StructField("name", name_schema),
                                      StructField("dob", DateType(), False),
                                      StructField("nationality", StringType(), False),
                                      StructField("url", StringType(), False)])

In [5]:
drivers_df = spark.read.json(r"E:\unused\Udemy\Spark_practice\raw\raw_files\drivers.json",schema=drivers_schema)
drivers_df.show(2)

+--------+---------+------+----+-----------------+----------+-----------+--------------------+
|driverId|driverRef|number|code|             name|       dob|nationality|                 url|
+--------+---------+------+----+-----------------+----------+-----------+--------------------+
|       1| hamilton|    44| HAM|{Lewis, Hamilton}|1985-01-07|    British|http://en.wikiped...|
|       2| heidfeld|  null| HEI| {Nick, Heidfeld}|1977-05-10|     German|http://en.wikiped...|
+--------+---------+------+----+-----------------+----------+-----------+--------------------+
only showing top 2 rows



In [6]:
drivers_trans_df = drivers_df.withColumnRenamed("driverId","driver_id").\
                             withColumnRenamed("driverRef","driver_ref").\
                             withColumn("Ingestion_date",current_timestamp()).\
                             withColumn("name",concat(col("name.forename"),lit(" "),col("name.surname")))

In [7]:
drivers_fnl_df = drivers_trans_df.drop("url")

In [8]:
drivers_fnl_df.write.mode("overwrite").parquet("processed/drivers")

In [9]:
df = spark.read(r'processed/drivers')

+----------------------------+--------------------------------------------------------------------+-------+
|col_name                    |data_type                                                           |comment|
+----------------------------+--------------------------------------------------------------------+-------+
|driver_id                   |int                                                                 |null   |
|driver_ref                  |string                                                              |null   |
|number                      |int                                                                 |null   |
|code                        |string                                                              |null   |
|name                        |string                                                              |null   |
|dob                         |date                                                                |null   |
|nationality                